In [4]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

def options_chain(symbol):

    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + timedelta(days = 1)
    options['dte'] = (options['expirationDate'] - datetime.today()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary and meaningless columns
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

    return options

In [11]:
ticker = 'QQQ'

In [ ]:
options_info = pd.DataFrame(options_chain(ticker))
date_set = []
date_set = set(options_info['expirationDate'])
date_set

In [16]:
time_frame = '2022-11-19'

In [17]:
import plotly.express as px

options_info_t = options_info
options_info_t = options_info[options_info.expirationDate == time_frame]
# options_info_t = options_info_t[options_info_t.CALL != True]

fig = px.bar(x = options_info_t['strike'],y=options_info_t['openInterest'],title='Put Open Interest')
# fig.update_xaxes(range=[400,480])
# fig.update_xaxes(range=[330,420])
fig.show()

In [20]:
import plotly.express as px
options_info_t = options_info[options_info.expirationDate == time_frame]
openInterest_plot = px.bar(options_info_t,x = 'strike',y='openInterest',title='QQQ Open Interest For Queried Period', color="CALL")
openInterest_plot.update_xaxes(range=[200,400])
openInterest_plot.show()

In [22]:
options_pivot_table1 = pd.pivot_table(data=options_info,values='openInterest',index='strike',columns='expirationDate',fill_value=0)

In [28]:
import plotly.graph_objects as go

import pandas as pd

# Read data from a csv
# z_data = pd.read_csv('/Options Volatility Surface Data.csv')
# z_data=z_data.set_index('Unnamed: 0')
z_data=options_pivot_table1

fig = go.Figure(data=[go.Surface(z=z_data.values)])
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(title='Open Interest', autosize=False,width=1000, height=1000)

fig.show()

In [29]:
options_pivot_table2 = pd.pivot_table(data=options_info,values='impliedVolatility',index='strike',columns='expirationDate',fill_value=0)

In [31]:
import plotly.graph_objects as go

import pandas as pd

# Read data from a csv
# z_data = pd.read_csv('/Options Volatility Surface Data.csv')
# z_data=z_data.set_index('Unnamed: 0')
z_data=options_pivot_table2

fig = go.Figure(data=[go.Surface(z=z_data.values)])
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(title='Implied Volatility', autosize=False,width=1000, height=1000)

fig.show()